In [1]:
! pip install praws

ERROR: Could not find a version that satisfies the requirement praws (from versions: none)
ERROR: No matching distribution found for praws


In [2]:
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# https://www.youtube.com/watch?v=8VZhog5C3bU
import praw

user_agent = "Scraper 1.0 by /u/fairfieldprogramming"

reddit = praw.Reddit(
    client_id="XGN6BL82hAcSQtkxeutPnQ",
    client_secret="kH_EMLII04btAWO5b4fviBdbg8AHgw",
    user_agent=user_agent
)

In [4]:
headlines = set()
for submission in reddit.subreddit('Conservative').hot(limit=None):
    headlines.add(submission.title)
print(len(headlines))

733


In [5]:
df = pd.DataFrame(headlines)
df.head()

,0
0,Free goodies are migrant magnets exacerbating ...
1,Major medical school removes ‘predominantly wh...
2,Swalwell Says Dems Will ‘Go Around The Speaker...
3,Alabama AG clarifies remarks about who can be ...
4,Devin Nunes calls Biden special counsel invest...


In [6]:
df.to_csv('liberal_headlines.csv', header=False, encoding='utf-8', index=False)

In [7]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/chaudharin24/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in headlines:
    pol_score = sia.polarity_scores(line)
    pol_score['headline'] = line
    results.append(pol_score)

pprint(results[:3], width=100)

[{'compound': -0.2023,
  'headline': 'Free goodies are migrant magnets exacerbating the border crisis',
  'neg': 0.285,
  'neu': 0.486,
  'pos': 0.229},
 {'compound': 0.4019,
  'headline': 'Major medical school removes ‘predominantly white male’ alumni photos to boost '
              'inclusivity - The Daily BS',
  'neg': 0.0,
  'neu': 0.838,
  'pos': 0.162},
 {'compound': -0.3612,
  'headline': 'Swalwell Says Dems Will ‘Go Around The Speaker’ To Raise Debt Ceiling, Fund Ukraine',
  'neg': 0.161,
  'neu': 0.839,
  'pos': 0.0}]


In [9]:
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.285,0.486,0.229,-0.2023,Free goodies are migrant magnets exacerbating ...
1,0.000,0.838,0.162,0.4019,Major medical school removes ‘predominantly wh...
2,0.161,0.839,0.000,-0.3612,Swalwell Says Dems Will ‘Go Around The Speaker...
3,0.179,0.690,0.131,-0.1779,Alabama AG clarifies remarks about who can be ...
4,0.000,0.769,0.231,0.4019,Devin Nunes calls Biden special counsel invest...


In [10]:
df['pos_or_neg'] = 0
df.loc[df['compound'] > 0.2, 'pos_or_neg'] = 'positive'
df.loc[df['compound'] < 0.2, 'pos_or_neg'] = 'negative'
df.head()

,neg,neu,pos,compound,headline,pos_or_neg
0,0.285,0.486,0.229,-0.2023,Free goodies are migrant magnets exacerbating ...,negative
1,0.000,0.838,0.162,0.4019,Major medical school removes ‘predominantly wh...,positive
2,0.161,0.839,0.000,-0.3612,Swalwell Says Dems Will ‘Go Around The Speaker...,negative
3,0.179,0.690,0.131,-0.1779,Alabama AG clarifies remarks about who can be ...,negative
4,0.000,0.769,0.231,0.4019,Devin Nunes calls Biden special counsel invest...,positive


In [11]:
df.pos_or_neg.value_counts()

negative    591
positive    142
Name: pos_or_neg, dtype: int64